In [1]:
import pandas as pd
import torch
import numpy as np

from sklearn.metrics import accuracy_score


from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, IntervalStrategy
from meat_dataset import MeatDataset
from utils import create_label_encoder

c:\Users\admin\PycharmProjects\test_task\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Чтение данных

In [2]:
train_df = pd.read_csv("train.csv", delimiter=';')
test_df = pd.read_csv("test.csv", delimiter=';')

In [3]:
labels_to_code, code_to_labels = create_label_encoder(pd.unique(train_df['mtype']))
print(labels_to_code)
print(code_to_labels)

{'Баранина': 0, 'Говядина': 1, 'Индейка': 2, 'Кура': 3, 'Свинина': 4, 'Цыпленок': 5}
{0: 'Баранина', 1: 'Говядина', 2: 'Индейка', 3: 'Кура', 4: 'Свинина', 5: 'Цыпленок'}


# Подготовка модели

In [4]:
tokenizer_path = 'cointegrated/rubert-tiny'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

model_path = 'cointegrated/rubert-tiny'
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(labels_to_code.keys()))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = MeatDataset(train_df, tokenizer, labels_to_code)
test_dataset = MeatDataset(test_df, tokenizer, labels_to_code)

In [6]:
SAVE_PATH = "model_data"
training_args = TrainingArguments(
    output_dir=SAVE_PATH,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10, 
    load_best_model_at_end=True,
    eval_strategy = IntervalStrategy.STEPS
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [7]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        
    }

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics
)

trainer.train()
tokenizer.save_pretrained(SAVE_PATH)
model.save_pretrained(SAVE_PATH)    

  0%|          | 0/16440 [00:00<?, ?it/s]c:\Users\admin\PycharmProjects\test_task\myenv\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  3%|▎         | 500/16440 [02:18<1:16:10,  3.49it/s]

{'loss': 0.8441, 'grad_norm': 1.4511138200759888, 'learning_rate': 1.9391727493917275e-05, 'epoch': 0.3}


                                                     
  3%|▎         | 500/16440 [02:43<1:16:10,  3.49it/s]

{'eval_loss': 0.3028925359249115, 'eval_accuracy': 0.9057177615571776, 'eval_runtime': 24.9513, 'eval_samples_per_second': 131.776, 'eval_steps_per_second': 16.472, 'epoch': 0.3}


  6%|▌         | 1000/16440 [05:09<1:14:10,  3.47it/s]

...

{'loss': 0.1017, 'grad_norm': 0.7016041278839111, 'learning_rate': 1.4525547445255475e-05, 'epoch': 2.74}


                                                      
 27%|██▋       | 4500/16440 [26:10<1:01:33,  3.23it/s]

{'eval_loss': 0.11438106745481491, 'eval_accuracy': 0.9628953771289538, 'eval_runtime': 26.0229, 'eval_samples_per_second': 126.35, 'eval_steps_per_second': 15.794, 'epoch': 2.74}


 27%|██▋       | 4500/16440 [26:11<1:09:29,  2.86it/s]


{'train_runtime': 1571.4383, 'train_samples_per_second': 83.681, 'train_steps_per_second': 10.462, 'train_loss': 0.21544961802164714, 'epoch': 2.74}


# Метрики, полученные на тестовых данных

In [9]:
predictions = trainer.predict(test_dataset)

100%|██████████| 411/411 [00:25<00:00, 16.17it/s]


In [10]:
predictions.metrics

{'test_loss': 0.11090797185897827,
 'test_accuracy': 0.9610705596107056,
 'test_runtime': 25.4973,
 'test_samples_per_second': 128.955,
 'test_steps_per_second': 16.119}